In [1]:
import numpy as np
from sklearn.neighbors import KDTree  
import math
import random

In [2]:
class PointContainer(list):
    def __new__(self, value, name = None, values = None):
        s = super(PointContainer, self).__new__(self, value)
        return s

In [3]:
def make_kd_tree(points, dim, i=0):
    if len(points) > 1:
        points.sort(key=lambda x: x[i])
        i = (i + 1) % dim
        half = len(points) >> 1
        return [
            make_kd_tree(points[: half], dim, i),
            make_kd_tree(points[half + 1:], dim, i),
            points[half]
        ]
    elif len(points) == 1:
        return [None, None, points[0]]

In [4]:
def add_node(kdtree, point, dim, i=0):
    if (kdtree == None):
        kdtree = [None, None, point]
        return
    if (kdtree[2][i] > point[i]):
        i = (i + 1) % dim
        add_node(kdtree[0], point, dim, i)
    else:
        i = (i + 1) % dim
        add_node(kdtree[1], point, dim, i)

In [5]:
def get_nearest(kd_node, point, dim, dist_func, i=0, best=None):
    if kd_node is not None:
        dist = dist_func(point, kd_node[2], dim)
        if not best:
            best = [dist, kd_node[2]]
        elif dist < best[0]:
            best[0], best[1] = dist, kd_node[2]

        dx = kd_node[2][i] - point[i] 
        #dx < 0 nghĩa là điểm P nằm phía bên phải của siêu phẳng x = kd_node[2][i] (nằm về phía x > kd_node[2][i])
        order = [] #thứ tự duyệt
        if dx < 0: 
            order = [1, 0] #duyệt nhánh phải trước, nhánh trái sau
        else: 
            order = [0, 1] #duyệt nhánh trái trước, nhánh phải sau
        
        #Duyệt nhánh cây ưu tiên trước
        get_nearest(kd_node[order[0]], point, dim, dist_func, (i + 1) % dim, best)
        #Nếu cần thì duyệt nhánh cây còn lại
        '''
        Giải thích:
            Nếu khoảng cách nhỏ nhất đã tìm ra (tính đến thời điểm hiện tại) bé hơn khoảng cách xét trên siêu phẳng i giữa điểm P và điểm kd_node[2] 
            thì ta bỏ qua việc xét điểm này (nghĩa là bỏ qua việc xét nhánh cây có gốc là điểm đó)
        '''
        dx = abs(dx)
        if dx < best[0]:
            get_nearest(kd_node[order[1]], point, dim, dist_func, (i + 1) % dim, best)
    return best

In [11]:
#Khởi tạo tập dữ liệu
#chọn số chiều là 3
dim = 3

def rand_point(dim):
    return [random.uniform(-1, 1) for d in range(dim)]

def dist_sq(a, b, dim):
    return math.sqrt(sum((a[i] - b[i]) ** 2 for i in range(dim)))

points = [PointContainer(rand_point(dim)) for x in range(10000)]
additional_points = [PointContainer(rand_point(dim))]

In [12]:
#Thử nghiệm K-d Tree mình vừa code với K-d Tree của Sklearn có ra cùng kết quả hông
kd_node = make_kd_tree(points, dim)
res1 = np.array([[get_nearest(kd_node, additional_points[0], dim, dist_sq)[0]]])
kd_tree_sklearn = KDTree(points, leaf_size = 1)
res2 = kd_tree_sklearn.query(additional_points, k=1)

print(res1)
print(res2)

[[0.04168985]]
(array([[0.04168985]]), array([[5800]]))


In [13]:
#Cũng là thử nghiệm nhưng mình for nhiều lần cho uy tín
for i in range(100):
    dim = 3

    def rand_point(dim):
        return [random.uniform(-1, 1) for d in range(dim)]

    def dist_sq(a, b, dim):
        return math.sqrt(sum((a[i] - b[i]) ** 2 for i in range(dim)))

    points = [PointContainer(rand_point(dim)) for x in range(10000)]
    additional_points = [PointContainer(rand_point(dim)) for x in range(50)]
    kd_node = make_kd_tree(points, dim)
    res1 = [np.array([[get_nearest(kd_node, point, dim, dist_sq)[0]]]) for point in additional_points]
    kd_tree_sklearn = KDTree(points, leaf_size = 1)
    res2 = [kd_tree_sklearn.query(np.reshape(point, (1, -1)), k=1)[0] for point in additional_points]
    if res1 != res2:
        print(i)

In [15]:
#Nếu đáp án của mình khác Sklearn thì nó in ra ngay
for i in range(50):
    if (res1[i] != res2[i]): 
        print(i)
        print(res1[i])
        print(res2[i])